<a href="https://colab.research.google.com/github/abakarova5665/NLTK/blob/main/Task_6_bot_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт необходимых библиотек

In [35]:
import pandas as pd
import re
import nltk
from nltk import word_tokenize, pos_tag
from nltk.stem import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances 
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
#библиотека для выгрузки файлов в гугл коллаб
from google.colab import files 

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Чтение файла


In [36]:
df = pd.read_excel('dialog_talk_agent.xlsx')

# Заполнение пустых ячеек значением из последней непустой ячейки 

In [37]:
df.ffill(axis = 0, inplace = True)
df.head(15)

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,I can help you work smarter instead of harder
3,Describe yourself,I can help you work smarter instead of harder
4,tell me about yourself,I can help you work smarter instead of harder
5,all about you,I can help you work smarter instead of harder
6,tell me some stuff about you,I can help you work smarter instead of harder
7,talk some stuff about you,I can help you work smarter instead of harder
8,talk about yourself,I can help you work smarter instead of harder
9,about yourself,I can help you work smarter instead of harder


### Перевод каждого слова в нижний регистр и вывод строк из столбца 'Context' без ненужных лишних знаков 

In [38]:
# def step1(text):
#   for word in text:
#     p = str(word).lower()
#     b = re.sub(r'[^a-z0-9]', ' ', p)
#     print(b)

# df1 = df.head(15)
# step1(df1['Context'])

### Разбиение строки на слова и тэгирование

In [39]:
#s = 'tell me about your personality'
#token = word_tokenize(s)
#pos_tag(token)

# Создание функции нормализации текста

In [40]:
def text_normalize(text):
  words = str(text).lower()  #перевод в нижний регистр
  words = re.sub(r'[^a-z0-9]', ' ', words) #выборока нужных символов
  token = word_tokenize(words)  #разбиение строки на слова
  tags = pos_tag(token)  #тэгирование слов
  lem_list = []   #создание пустого списка
  lemm = wordnet.WordNetLemmatizer()  #лемматизация
  for word, tag in tags:  #перебираем каждое слово
    if tag.startswith('V'):  #если тэг начинается с V, J, R or N
      pos_val = 'v'         
    elif tag.startswith('J'):
      pos_val = 'a'
    elif tag.startswith('R'):
      pos_val = 'r'
    else:
      pos_val = 'n'
    lemm_token = lemm.lemmatize(word, pos_val)   #лемматизация
    lem_list.append(lemm_token)
  return ' '.join(lem_list)

In [41]:
df['Lemmatize'] = df['Context'].apply(text_normalize)
df.head(25)

,Context,Text Response,Lemmatize
0,Tell me about your personality,Just think of me as the ace up your sleeve.,tell me about your personality
1,I want to know you better,I can help you work smarter instead of harder,i want to know you good
2,Define yourself,I can help you work smarter instead of harder,define yourself
3,Describe yourself,I can help you work smarter instead of harder,describe yourself
4,tell me about yourself,I can help you work smarter instead of harder,tell me about yourself
5,all about you,I can help you work smarter instead of harder,all about you
6,tell me some stuff about you,I can help you work smarter instead of harder,tell me some stuff about you
7,talk some stuff about you,I can help you work smarter instead of harder,talk some stuff about you
8,talk about yourself,I can help you work smarter instead of harder,talk about yourself
9,about yourself,I can help you work smarter instead of harder,about yourself


In [42]:
stop = stopwords.words('english')

# Считаем углы между векторами, чем меньше угол, тем более подходящий ответ будет подобран

In [43]:
cv = CountVectorizer()
X = cv.fit_transform(df['Lemmatize']).toarray()
features = cv.get_feature_names()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head(15)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,21,abort,about,absolutely,abysmal,actually,adore,advice,advise,affirmative,...,year,yeh,yep,yes,yet,you,your,yours,yourself,yup
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
7,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# Вывод запроса без стоп-слов

In [44]:
Quest = 'Can you help me?'
Q = []
a = Quest.split()
for word in a:
  if word not in stop:
    Q.append(word)
Q = ' '.join(Q)
Q

'Can help me?'

# Конвертация векторов в массив и вывод

In [45]:
Q = text_normalize(Q)
Q_bow = cv.transform([Q]).toarray()
Q_bow

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Вывод косинусного расстояния в новом столбце 'Simil'

In [46]:
cosine_value = 1 - pairwise_distances(df_bow, Q_bow, metric = 'cosine')
df['Simil'] = cosine_value
df.head(25)

,Context,Text Response,Lemmatize,Simil
0,Tell me about your personality,Just think of me as the ace up your sleeve.,tell me about your personality,0.258199
1,I want to know you better,I can help you work smarter instead of harder,i want to know you good,0.000000
2,Define yourself,I can help you work smarter instead of harder,define yourself,0.000000
3,Describe yourself,I can help you work smarter instead of harder,describe yourself,0.000000
4,tell me about yourself,I can help you work smarter instead of harder,tell me about yourself,0.288675
5,all about you,I can help you work smarter instead of harder,all about you,0.000000
6,tell me some stuff about you,I can help you work smarter instead of harder,tell me some stuff about you,0.235702
7,talk some stuff about you,I can help you work smarter instead of harder,talk some stuff about you,0.000000
8,talk about yourself,I can help you work smarter instead of harder,talk about yourself,0.000000
9,about yourself,I can help you work smarter instead of harder,about yourself,0.000000


# Вывод запроса и ответа нат запрос

In [47]:
ans_ind = cosine_value.argmax()
ans = df['Text Response'].loc[ans_ind]
print('Quest\n', Quest, '\n', 'Answer\n', ans)

Quest
 Can you help me? 
 Answer
 I'm glad to help. What can I do for you?
